In [2]:
from keras.datasets import imdb
import copy
top_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)


Using TensorFlow backend.


17465344/17464789 [==============================] - 3s 0us/step


In [7]:
json_dicionario = imdb.get_word_index()
def traduzir(revisao):
    for palavra in revisao:
        if(palavra in json_dicionario.items().values()):
            print("achou")
teste = copy.deepcopy(x_train[0])
traduzir(teste)
print("fim")


    

AttributeError: 'dict_items' object has no attribute 'values'